In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from SALib.sample import saltelli
from SALib.analyze import sobol

import sys
import os
import ast
sys.path.append(os.path.abspath("../.."))

from Modules.Models import ModelWrapper
from Modules.EvolutionModules import Individual

# --- Configurações ---
CSV_FILE = "cmaes_results.csv"  # CSV com os resultados do CMA-ES/DE
MODEL_SIZE = 19                 # Número de parâmetros do modelo
FITNESS_COLUMN = "SQUARED_Fitness"
CALC_SECOND_ORDER = True

# --- Carrega o modelo uma vez ---
model = ModelWrapper.GRN5()  # <-- usa o modelo já carregado com dados

# --- Função para extrair parâmetros do CSV ---
def extract_params(best_ind_str):
    try:
        # pega só a parte de 'coeffs'
        start = best_ind_str.find("coeffs=") + len("coeffs=")
        end = best_ind_str.rfind("}, ind_size")
        coeffs_str = best_ind_str[start:end] + "}"  # fecha dicionário
        
        # remove "val=" para que ast.literal_eval funcione
        coeffs_str = coeffs_str.replace("val=", "")
        
        coeffs = ast.literal_eval(coeffs_str)
        
        params = []
        for gene in ['A', 'B', 'C', 'D', 'E']:
            for target, values in coeffs[gene].items():
                if isinstance(values, dict):
                    for key in ['n','k','tau']:
                        if key in values:
                            params.append(values[key])
                elif target == 'tau':
                    params.append(values)
        return np.array(params)
    except Exception as e:
        print("Erro ao extrair parâmetros:", e)
        return None

# --- Lê CSV e extrai parâmetros e fitness ---
df = pd.read_csv(CSV_FILE)
param_list = []
fitness_list = []

print("Extraindo parâmetros válidos do CSV...")
for _, row in tqdm(df.iterrows(), total=len(df)):
    params = extract_params(row['best_ind'])
    if params is None or len(params) != MODEL_SIZE:
        continue
    fitness = row[FITNESS_COLUMN]
    if np.isfinite(fitness):
        param_list.append(params)
        fitness_list.append(fitness)

param_values_csv = np.array(param_list)
fitness_values_csv = np.array(fitness_list)

print(f"Total de amostras válidas: {len(fitness_values_csv)}")

# --- Define bounds para Sobol com base no CSV ---
bounds = [[float(np.min(param_values_csv[:, i])), float(np.max(param_values_csv[:, i]))]
          for i in range(MODEL_SIZE)]

problem = {
    'num_vars': MODEL_SIZE,
    'names': [f'param_{i}' for i in range(MODEL_SIZE)],
    'bounds': bounds
}

# --- Gera amostras Saltelli dentro dos limites observados ---
N = 1024  # base de amostras
param_values_sobol = saltelli.sample(problem, N, calc_second_order=CALC_SECOND_ORDER)
print(f"Total de amostras Saltelli: {len(param_values_sobol)}")

# --- Calcula fitness para cada amostra ---
Y_sobol = []
print("Calculando fitness para cada amostra...")
for params in tqdm(param_values_sobol):
    ind = Individual.list_to_ind(params, model)  # <-- usa o modelo já carregado
    ind.calculate_fitness(solver='ODEINT', error='MSE')
    Y_sobol.append(ind.fitness)

Y_sobol = np.array(Y_sobol)

# --- Executa análise Sobol ---
print("\nExecutando análise Sobol...")
Si = sobol.analyze(problem, Y_sobol, calc_second_order=CALC_SECOND_ORDER, print_to_console=True)

# --- Resultados ---
print("\n--- Resultados de Sensibilidade ---")
print("S1 (efeito direto):", Si['S1'])
print("ST (efeito total):", Si['ST'])
if CALC_SECOND_ORDER:
    print("S2 (interações pares):", Si['S2'])


c:\Users\anabi\OneDrive\Documentos\# BOLSA GRN\Códigos\BOLSA-GRN\PythonCode\Modules\Helpers.py:27: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(filename, delim_whitespace=True, header=None, names=['t'] + labels)


Extraindo parâmetros válidos do CSV...


100%|██████████| 20/20 [00:00<00:00, 2050.15it/s]
C:\Users\anabi\AppData\Local\Temp\ipykernel_21652\2406849110.py:83: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values_sobol = saltelli.sample(problem, N, calc_second_order=CALC_SECOND_ORDER)


Total de amostras válidas: 20
Total de amostras Saltelli: 40960
Calculando fitness para cada amostra...


  1%|▏         | 533/40960 [00:01<01:56, 345.55it/s]C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_odepack_py.py:248: ODEintWarning: Repeated error test failures (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
100%|██████████| 40960/40960 [01:59<00:00, 342.29it/s]
C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))



Executando análise Sobol...
                ST   ST_conf
param_0   0.044411  0.044208
param_1   0.131320  0.063989
param_2   0.053821  0.035975
param_3   0.120920  0.134808
param_4   0.261091  0.144709
param_5   0.146448  0.123366
param_6   0.167904  0.107945
param_7   0.142888  0.066210
param_8   0.251810  0.310886
param_9   0.102850  0.135851
param_10  0.133778  0.086687
param_11  0.092419  0.064242
param_12  0.043797  0.040441
param_13  0.291885  0.147118
param_14  0.054159  0.083717
param_15  0.115099  0.083044
param_16  0.196920  0.315774
param_17  0.172153  0.155509
param_18  0.074450  0.060623
                S1   S1_conf
param_0   0.004060  0.011960
param_1   0.045208  0.032542
param_2  -0.005576  0.021245
param_3  -0.006055  0.015350
param_4   0.093858  0.057361
param_5   0.017035  0.022689
param_6   0.215800  0.208909
param_7   0.042180  0.031762
param_8   0.003354  0.020894
param_9  -0.002362  0.013801
param_10  0.047028  0.027538
param_11  0.000577  0.021047
param_12  0.01